# IGN Scraper for PlayStation Posts 🎮

**https://www.ign.com/robots.txt**

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd

In [39]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print("Loading more comments...")
        time.sleep(5) 
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        print("Soup created")  
        date_elements = soup.find_all('div', class_='interface jsx-153568585 jsx-957202555 item-subtitle small')
        print(f"Found {len(date_elements)} date elements")  
        for date_element in date_elements:
            text = date_element.get_text(strip=True).split('-')[0].strip()  
            print(f"Checking date: {text}")  
            if '2022' in text:  
                print("Date found, stopping scroll")  
                return
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  
        last_height = new_height  

In [40]:
def extract_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all('div', class_='content-item jsx-1409608325 row divider')
    data = []
    for item in items:
        title = item.find('span', class_='item-title bold').get_text(strip=True) if item.find('span', class_='item-title bold') else ''
        date = item.find('div', class_='interface jsx-153568585 jsx-957202555 item-subtitle small').get_text(strip=True).split('-')[0].strip() if item.find('div', class_='interface jsx-153568585 jsx-957202555 item-subtitle small') else 'Null'
        author_link = item.find('object', title='Author Link')
        author = author_link.find('a').get_text(strip=True) if author_link and author_link.find('a') else 'Null'
        comment_text = item.find('div', class_='caption jsx-789695853 comment-count data small').get_text(strip=True) if item.find('div', class_='caption jsx-789695853 comment-count data small') else ''
        comment_count = ''.join(filter(str.isdigit, comment_text)) 
        comment_count = int(comment_count) if comment_count else 0
        related_content_link = item.find('object', title='Related Content Link')
        game_title = related_content_link.find('a').get_text(strip=True) if related_content_link and related_content_link.find('a') else 'Null'
        link = item.find('a', href=True)['href'] if item.find('a', href=True) else 'Null'
        data.append({
            'Date': date,
            'Title': title,
            'Author': author,
            'Game Title': game_title,
            'Comment Count': comment_count,
            'Link': 'https://www.ign.com' + link
        })
    return data

In [41]:
def scrape_ign():
    driver = webdriver.Chrome()
    driver.get("https://www.ign.com/?filter=playstation")
    scroll_to_bottom(driver)
    scraped_data = extract_data(driver)
    driver.quit()
    return pd.DataFrame(scraped_data)

In [ ]:
df = scrape_ign()
print(df)

---
---
---

# 🔍 Inspection Area 🥼 🤓

In [ ]:
from itables import show 

show(df)

In [ ]:
print(df.head(2))

print(df.shape)

In [ ]:
# to csv
# df.to_csv('RAW_ign_PS_titles.csv', index=False)